In [ ]:
import scvelo as scv
import loompy
import igraph as ig
import anndata as ad
import pandas as pd
import re
import scanpy as sc 
import matplotlib.pyplot as plt 
import numpy as np 
import scipy as sp
import seaborn as sb
import harmonypy

adata = sc.read_h5ad(filename = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/velocyto/scvelo_output/use/deterministic/deterministic_all_velos_neighbors_pca.h5ad")

In [ ]:
def grouped_obs_mean(adata, group_key, layer=None, gene_symbols=None):
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X
    if gene_symbols is not None:
        new_idx = adata.var[idx]
    else:
        new_idx = adata.var_names

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel(X.mean(axis=0, dtype=np.float64))
    return out

In [1]:
# define stage of interest, choose between iPSC, mesendoderm, predefendoderm, defendoderm

celltype = "iPSC"

DE_df = sc.get.rank_genes_groups_df(adata, group = celltype)
DE_names = DE_df['names'].tolist()

geneswodoubles = list(dict.fromkeys(DE_names))
adata = adata[:, geneswodoubles]
adata

In [ ]:
#write phenotype file 
blub = pd.DataFrame()
cells = pd.DataFrame()
sample_ids = []

cells = adata[adata.obs['celltype'] == celltype, :]
blub = grouped_obs_mean(cells, ['donor', 'exp_batch'])
blub = blub.T

for name in blub.index:
   item = name[0] + "-" + name[1]
   sample_ids.append(item)

#print(sample_ids)
blub.insert(0, 'feature_id', sample_ids)
blub = blub.T
out_path = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/vqtls/data/velocyto/expression/celltypes/" + celltype + "/input/"
blub.to_csv(out_path + "pheno_file_expression_" + celltype + '_2000DE.txt', header=False, index=True, sep='\t', mode='a')

In [ ]:
#link file alias sample mapping file
#maps genotype names to sample names 

donormeta = pd.read_csv('/icgc/dkfzlsdf/analysis/B260/projects/HipSci/openAccess/endoderm_differentation/metadata/HipSci_donor_annotation.tsv', sep = ';', header = 0)
donormeta.head(10)

subdf = pd.DataFrame()
subdf = donormeta[['Sample2', 'DonorLine']]
subdf['sample_map'] = "zero"
subdf = subdf.set_index('Sample2')

for index, row in subdf.iterrows():
    donor = row['DonorLine']
    for sample_id in sample_ids:
        if donor in sample_id:
            #print(True)
            subdf.loc[index, 'sample_map'] = sample_id
subdf["DonorLine"] = subdf.index

#subdf.to_csv(out_path + "sample_mapping_file_" + celltype + '.txt', header=False, index=False, sep='\t', mode='a')


In [ ]:
#make feature annotation file 
feature_anno_file = adata.var
feature_anno_file.head(10)

In [ ]:
feature_anno_file.index = range(len(feature_anno_file))
feature_anno_file.loc[-1] = ["feature_id", "chromosome", "end", "start", "strand"]  # adding a row
feature_anno_file.index = feature_anno_file.index + 1  # shifting index
feature_anno_file = feature_anno_file.sort_index() 
feature_anno_file = feature_anno_file[['Accession', "Chromosome", "Start", "End"]]
feature_anno_file.to_csv('/omics/groups/OE0540/internal/users/ruehle/rnavelocity/vqtls/data/velocyto/expression/feature_anno_Ensembl75.txt', header=False, index=False, sep='\t', 